In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/summary/cleaned_2.csv
/kaggle/input/summary/cleaned_5.csv
/kaggle/input/summary/cleaned_1.csv
/kaggle/input/summary/cleaned_4.csv
/kaggle/input/summary/cleaned_3.csv
/kaggle/input/dataset-restaurant/restaurant_categories_qa.csv
/kaggle/input/dataset-restaurant/restaurant_price_qa.csv
/kaggle/input/dataset-restaurant/restaurant_features_qa.csv


In [2]:
from huggingface_hub import login

login(token='hf_xxxxxxxxxxxxxxxx')

In [3]:
# Download dataset from Kaggle
!kaggle datasets download -d yelp-dataset/yelp-dataset --force

# Unzip the dataset, overwriting existing files without prompt
!unzip -o yelp-dataset.zip -d yelp-dataset
!ls yelp-dataset

import json
import pandas as pd

data_file = open("yelp-dataset/yelp_academic_dataset_business.json")
data = []
for line in data_file:
    data.append(json.loads(line))
checkin_df = pd.DataFrame(data)
data_file.close()

Dataset URL: https://www.kaggle.com/datasets/yelp-dataset/yelp-dataset
License(s): other
100%|██████████████████████████████████████▊| 4.06G/4.07G [00:20<00:00, 239MB/s]
100%|███████████████████████████████████████| 4.07G/4.07G [00:20<00:00, 218MB/s]
Archive:  yelp-dataset.zip
  inflating: yelp-dataset/Dataset_User_Agreement.pdf  
  inflating: yelp-dataset/yelp_academic_dataset_business.json  
  inflating: yelp-dataset/yelp_academic_dataset_checkin.json  
  inflating: yelp-dataset/yelp_academic_dataset_review.json  
  inflating: yelp-dataset/yelp_academic_dataset_tip.json  
  inflating: yelp-dataset/yelp_academic_dataset_user.json  
Dataset_User_Agreement.pdf	     yelp_academic_dataset_review.json
yelp_academic_dataset_business.json  yelp_academic_dataset_tip.json
yelp_academic_dataset_checkin.json   yelp_academic_dataset_user.json


In [4]:
import pandas as pd
import os

file_path = 'yelp-dataset'  

json_files = [
    'yelp_academic_dataset_review.json'
]

# Function to load JSON data in chunks
def load_json_in_chunks(file, chunksize=10000):
    chunk_list = []
    for chunk in pd.read_json(file, lines=True, chunksize=chunksize):
        chunk_list.append(chunk)
    return pd.concat(chunk_list, ignore_index=True)

dataframes = {}

for json_file in json_files:
    full_path = os.path.join(file_path, json_file)
    if os.path.exists(full_path):  # Check if the file exists before loading
        df = load_json_in_chunks(full_path)
        dataframes[json_file] = df
    else:
        print(f"File not found: {full_path}")

# Access the DataFrame for 'yelp_academic_dataset_review.json'
review_df = dataframes.get('yelp_academic_dataset_review.json')
if review_df is not None:
    print("DataFrame loaded successfully.")
else:
    print("DataFrame not loaded. File might be missing or path is incorrect.")

DataFrame loaded successfully.


In [5]:
review_df = review_df.loc[review_df.business_id.isin(checkin_df.loc[
                                                     ((checkin_df['state']=='LA')&(checkin_df['is_open']==1))
                                                     &
                                                     (checkin_df.categories.apply(lambda x: str(x).lower()).str.contains('restaurant') | checkin_df.categories.apply(lambda x: str(x).lower()).str.contains('food'))
                                                     ,'business_id'])]

unique_business_ids = review_df['business_id'].nunique()
print("Number of unique business IDs:", unique_business_ids)

# Sample up to 50 reviews per business
sampled_reviews = review_df.groupby('business_id').apply(lambda x: x.sample(min(50, len(x)))).reset_index(drop=True)

# Aggregate sampled reviews for each business
df_review_agg = sampled_reviews.groupby('business_id')['text'].agg('. '.join).reset_index()
df_review_agg.head()

Number of unique business IDs: 3027


/tmp/ipykernel_70/1626260834.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_reviews = review_df.groupby('business_id').apply(lambda x: x.sample(min(50, len(x)))).reset_index(drop=True)


,business_id,text
0,--zb12mw2YK-7j6UaHzm8w,I was always a fan of Manuel's Hot Tamales fro...
1,-0__F9fnKt8uioCKztF5Ww,"For as unsold as I am on the hotel, the restau..."
2,-0ltw8--HLuulPyOSspqAQ,My daughter and her family stayed here for the...
3,-1XSzguS6XLN-V6MVZMg2A,I have been lucky enough to visit New Orleans ...
4,-4x3pVUUsfWmKEilWKsOZQ,The food and service was fantastic. Dickeys is...


In [6]:
import pandas as pd
import os

# Path
dataset_path = '/kaggle/input/summary/'

# List of files
file_names = ['cleaned_1.csv', 'cleaned_2.csv', 'cleaned_3.csv', 'cleaned_4.csv', 'cleaned_5.csv']

dataframes = []

for file_name in file_names:
    file_path = os.path.join(dataset_path, file_name)  # Construct full file path
    if os.path.exists(file_path):  # Check if the file exists
        df = pd.read_csv(file_path)
        dataframes.append(df)
        print(f"Successfully loaded: {file_name}")
    else:
        print(f"File not found: {file_name}")

# Combine all DataFrames
if dataframes:
    combined_df = pd.concat(dataframes, ignore_index=True)
    print("Combined DataFrame:")
    print(combined_df.head())
else:
    print("No files were successfully loaded.")

Successfully loaded: cleaned_1.csv
Successfully loaded: cleaned_2.csv
Successfully loaded: cleaned_3.csv
Successfully loaded: cleaned_4.csv
Successfully loaded: cleaned_5.csv
Combined DataFrame:
              business_id                          assistant_summary_cleaned
0  --zb12mw2YK-7j6UaHzm8w  Here's a 50-word summary of food, service, onl...
1  -0__F9fnKt8uioCKztF5Ww  Here is  a summary of your experience at PISCO...
2  -0ltw8--HLuulPyOSspqAQ  Here's an 80-word summary focusing only on the...
3  -1XSzguS6XLN-V6MVZMg2A  The reviewer's review of ReBirth restaurant hi...
4  -4x3pVUUsfWmKEilWKsOZQ  The reviewer's third visit to this BBQ joint e...


In [7]:
business_df = checkin_df[['business_id','name','address','city','state','postal_code','categories','stars','review_count']]
business_df.loc[:, 'rating'] = ((business_df['stars'] * business_df['review_count']) / business_df['review_count']).round(2)
final_df = combined_df.merge(business_df, how='inner', on='business_id')[['name','address','city','state','postal_code','categories','rating','assistant_summary_cleaned']]\
                      .rename(columns={'assistant_summary_cleaned':'summary'})

/tmp/ipykernel_70/2195657229.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  business_df.loc[:, 'rating'] = ((business_df['stars'] * business_df['review_count']) / business_df['review_count']).round(2)


In [8]:
file_path1 = "/kaggle/input/dataset-restaurant/restaurant_categories_qa.csv"
file_path2 = "/kaggle/input/dataset-restaurant/restaurant_features_qa.csv"
file_path3 = "/kaggle/input/dataset-restaurant/restaurant_price_qa.csv"
# Read the CSV file into a pandas DataFrame
categories_qa= pd.read_csv(file_path1)
features_qa= pd.read_csv(file_path2)
price_qa= pd.read_csv(file_path3)
categories_qa_question = categories_qa.query('type_qna=="cate_base"')
categories_qa_answer = categories_qa.query('type_qna=="review_summary"')

In [9]:
pd.set_option('display.width', None)  
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
categories_qa_question.head(1)

,name,address,categories,type_qna,text
0,Copper Vine,1001 Poydras St New OrleansLA70112,"Nightlife, Pubs, Event Planning & Services, Wine Bars, Bars, Gastropubs, Restaurants, Venues & Event Spaces",cate_base,"<inst>Could you recommend me a restuarnt that is known as these keywords - Nightlife, Pubs, Event Planning & Services, Wine Bars, Bars, Gastropubs, Restaurants, Venues & Event Spaces?</inst>The restaurant that you are looking for is Copper Vine and they are known also as"


In [10]:
import re

def reformat_features_qa(value):
    if '<inst>' in value:  # Check if the format is correct
        # Remove the inst tags and split the content
        content = value.replace('<inst>', '').replace('</inst>', '').strip()
        question, answer = content.split(')', 1)  # Split at the closing parenthesis for Q&A
        question = question.strip() + ')'  # Ensure the question ends correctly
        answer = answer.strip()
        
        # Reformat to the desired structure
        reformatted = f"<inst>question: {question}</inst> Answer: {answer}<EOS>"
        return reformatted
    return value  # If no '<inst>', return value unchanged

def reformat_price_qa(value):
    if '<inst>' in value:  # Check if the format is correct
        # Remove the <inst> and </inst> tags and split the content
        content = value.replace('<inst>', '').replace('</inst>', '').strip()
        
        # Find where the question ends (after the closing question mark) and the answer starts
        question_end = content.find('?') + 1  # Find the question mark
        question = content[:question_end].strip()  # Extract the question part
        answer = content[question_end:].strip()  # The rest is the answer
        
        # Reformat to the desired structure
        reformatted = f"<inst>question: {question}</inst> Answer: {answer}<EOS>"
        return reformatted
    return value  # If no '<inst>', return value unchanged

def extract_keywords(text):
    # Remove unwanted phrases and symbols
    cleaned_text = re.sub(
        r"(\*\*.*?\*\*|Distinctive Keywords|Restaurant Keywords|Distinctive and Descriptive Keywords|Distinctive Features|Name:|Words:|Pros:|Cons:|and|\+|\\n|[\*\-\d\.])", 
        "", 
        text
    )
    # Remove extra whitespaces and split by words
    keywords = re.split(r"[,\s]+", cleaned_text.strip())
    # Return unique, non-empty keywords as a string
    return ", ".join(filter(None, map(str.strip, keywords)))
    
# Apply the function to the column
features_qa['features_Q_A'] = features_qa['features_Q_A'].apply(reformat_features_qa)
price_qa['price_Q_A'] = price_qa['price_Q_A'].apply(reformat_price_qa)
categories_qa_answer["Keywords"] = categories_qa_answer["text"].apply(extract_keywords)

/tmp/ipykernel_70/1824373783.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  categories_qa_answer["Keywords"] = categories_qa_answer["text"].apply(extract_keywords)


In [11]:
pd.set_option('display.width', None)  
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
price_qa.head(2)

,name,address,categories,price_Q_A
0,Copper Vine,1001 Poydras St,"Nightlife, Pubs, Event Planning & Services, Wine Bars, Bars, Gastropubs, Restaurants, Venues & Event Spaces",<inst>question: How much does the menu cost at the restaurant?</inst> Answer: This restaurant's price range is $$<EOS>
1,Mahony's Po-Boys & Seafood,901 Iberville St,"Restaurants, Seafood, Cajun/Creole",<inst>question: How much does the menu cost at the restaurant?</inst> Answer: This restaurant's price range is $$<EOS>


In [12]:
categories_tb = pd.merge(categories_qa_question, categories_qa_answer[['name','address','categories','Keywords']], on=['name','address','categories'], how='inner')

In [13]:
categories_tb["modified_text"] = categories_tb["text"].apply(
    lambda x: x.replace("<inst>", "<inst>question: ").replace("</inst>", "</inst> Answer: ")
)

# Combine modified_text and Keywords into the categories_Q_A column
categories_tb["categories_Q_A"] = categories_tb.apply(
    lambda row: f"{row['modified_text']} {row['Keywords']}<EOS>", axis=1
)
# Display the result
categories_tb['street_address'] = categories_tb['address'].str.extract(r"^(.*)\sNew Orleans")
categories_tb= categories_tb[['name','street_address','categories','categories_Q_A']]
categories_tb.rename(columns={'street_address':'address'}, inplace=True)

In [14]:
categories_tb.head()

,name,address,categories,categories_Q_A
0,Copper Vine,1001 Poydras St,"Nightlife, Pubs, Event Planning & Services, Wine Bars, Bars, Gastropubs, Restaurants, Venues & Event Spaces","<inst>question: Could you recommend me a restuarnt that is known as these keywords - Nightlife, Pubs, Event Planning & Services, Wine Bars, Bars, Gastropubs, Restaurants, Venues & Event Spaces?</inst> Answer: The restaurant that you are looking for is Copper Vine and they are known also as Copper, Vine, New, Orleans, Restaurant, Fine, dining, Wine, bar, Bottomless, brunch, Fine, cuisine, Upscale, atmosphere, Friendly, service, Excellent, food, Good, drinks, Unique, ambiance, Excellent, food, Great, atmosphere, Friendly, service, Excellent, wine, selection, Unique, ambiance<EOS>"
1,Mahony's Po-Boys & Seafood,901 Iberville St,"Restaurants, Seafood, Cajun/Creole","<inst>question: Could you recommend me a restuarnt that is known as these keywords - Restaurants, Seafood, Cajun/Creole?</inst> Answer: The restaurant that you are looking for is Mahony's Po-Boys & Seafood and they are known also as New, Orleans, PoBoys, Fried, Shrimp, Etouffee, Crawfish, Crab, Étouffée, Shrimp, Remi, PoBoy, Lobster, Oyster, PoBoy, Catfish, Gumbo<EOS>"
2,Nesbit's Magazine Street Market,301 Magazine St,"Convenience Stores, Food, Grocery, Coffee & Tea, Beer, Wine & Spirits","<inst>question: Could you recommend me a restuarnt that is known as these keywords - Convenience Stores, Food, Grocery, Coffee & Tea, Beer, Wine & Spirits?</inst> Answer: The restaurant that you are looking for is Nesbit's Magazine Street Market and they are known also as Convenient, location, Affordable, prices, Wide, selection, of, wine, snacks, Friendly, staff, Clean, store, Good, local, options, Southern, hospitality, Oldfashioned, business, style, Perfect, spot, for, necessities, Affordable, prices, Wide, selection, of, wine, snacks, Friendly, staff, Good, local, options, Clean, store<EOS>"
3,Taste of Nawlins,504 Madison St,"Historical Tours, Tours, Hotels & Travel, Food Tours, Walking Tours","<inst>question: Could you recommend me a restuarnt that is known as these keywords - Historical Tours, Tours, Hotels & Travel, Food Tours, Walking Tours?</inst> Answer: The restaurant that you are looking for is Taste of Nawlins and they are known also as Food, Tour, New, Orleans, French, Quarter, Gumbo, Jambalaya, Praline, Brisket, Po', Boy, Muffuletta, Bread, Pudding, Taste, of, New, Orleans, Groupon, Restaurant, Guide, Tour, Guide, New, Orleans, Tour, Food, History<EOS>"
4,Blaze Pizza,611 O'keefe Ave,"Pizza, Fast Food, Gluten-Free, Restaurants, Salad","<inst>question: Could you recommend me a restuarnt that is known as these keywords - Pizza, Fast Food, Gluten-Free, Restaurants, Salad?</inst> Answer: The restaurant that you are looking for is Blaze Pizza and they are known also as Pizza, Blaze, Pizza, Fast, food, Affordable, Quick, service, Customizable, toppings, Thin, crust, Woodburning, oven, Assembly, line, model, Unlimited, toppings, Fresh, ingredients, Good, customer, service, Friendly, staff, Clean, atmosphere, Great, value, Delicious, pizzas, Good, prices<EOS>"


In [15]:
# location-related questions
location_questions = [
    "Where is {name} located?",
    "Can you tell me the location of {name}?",
    "What is the address of {name}?",
    "How can I find {name}?"
]

# Adding the 'Q_A_loc' column by rotating the location questions and including details
final_df['Q_A_loc'] = final_df.apply(
    lambda x: (
        f"<inst>question: {location_questions[x.name % len(location_questions)].format(name=x['name'])} </inst> "  # Rotate through questions
        f"Answer: This restaurant is located at {x['address']}, {x['city']}, {x['state']} {x['postal_code']}<EOS> "
    ),
    axis=1
)

In [16]:
# list of questions about restaurant information
questions = [
    "What is good about {name}?",
    "What’s the general opinion on {name}?",
    "How do customers describe {name}?",
    "What can you tell me about {name}?"
]

# Adding the 'Q_A_Info' column by rotating the questions and including details
final_df['Q_A_Info'] = final_df.apply(
    lambda x: (
        f"<inst>question: {questions[x.name % len(questions)].format(name=x['name'])} </inst> "  # Rotate through questions
        f"Answer: {x['summary']}. {x['categories']}. This restaurant has a rating of {round(x['rating'], 1)} stars. "
        f"It is located in the zip code {x['postal_code']}.<EOS>"
    ),
    axis=1
)

In [17]:
pd.set_option('display.width', None)  
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
final_df.head(2)

,name,address,city,state,postal_code,categories,rating,summary,Q_A_loc,Q_A_Info
0,Billy Wright's Manuel's Style Hot Tamales,Robert E. Lee & Canal Blvd,New Orleans,LA,70124,"Food, Street Vendors",4.5,"Here's a 50-word summary of food, service, only: Manuel's-style tames are a N'awlins classic. Fresh, delicious, & inexpensive. Owners Billy and his wife make them with love, using traditional recipes. Perfect for parties or just a quick bite. Try their tams with beer for an authentic experience. A must-try for anyone visiting NOLA!","<inst>question: Where is Billy Wright's Manuel's Style Hot Tamales located? </inst> Answer: This restaurant is located at Robert E. Lee & Canal Blvd, New Orleans, LA 70124<EOS>","<inst>question: What is good about Billy Wright's Manuel's Style Hot Tamales? </inst> Answer: Here's a 50-word summary of food, service, only: Manuel's-style tames are a N'awlins classic. Fresh, delicious, & inexpensive. Owners Billy and his wife make them with love, using traditional recipes. Perfect for parties or just a quick bite. Try their tams with beer for an authentic experience. A must-try for anyone visiting NOLA!. Food, Street Vendors. This restaurant has a rating of 4.5 stars. It is located in the zip code 70124.<EOS>"
1,Piscobar,914 Union St,New Orleans,LA,70112,"Cafes, Nightlife, Cocktail Bars, Peruvian, Restaurants, Vegan, Bars",4.0,"Here is a summary of your experience at PISCOBAR in 250 words: You had an overall good experience, particularly with your server, Sean, at Catahoulah Hotel, where you stayed. However, you encountered issues with service, such as long wait times, unhelpful staff (Chelsea and Tristan), poor food quality (e.g., underseasoned and burnt dishes), high prices (especially for non-vegetarian options), limited vegan menu options, inconsistent menu updates,","<inst>question: Can you tell me the location of Piscobar? </inst> Answer: This restaurant is located at 914 Union St, New Orleans, LA 70112<EOS>","<inst>question: What’s the general opinion on Piscobar? </inst> Answer: Here is a summary of your experience at PISCOBAR in 250 words: You had an overall good experience, particularly with your server, Sean, at Catahoulah Hotel, where you stayed. However, you encountered issues with service, such as long wait times, unhelpful staff (Chelsea and Tristan), poor food quality (e.g., underseasoned and burnt dishes), high prices (especially for non-vegetarian options), limited vegan menu options, inconsistent menu updates,. Cafes, Nightlife, Cocktail Bars, Peruvian, Restaurants, Vegan, Bars. This restaurant has a rating of 4.0 stars. It is located in the zip code 70112.<EOS>"


In [18]:
final_df = (final_df
            .merge(price_qa[['name','address','categories','price_Q_A']], on=['name','address','categories'], how='left')
            .merge(features_qa[['name','address','categories','features_Q_A']], on=['name','address','categories'],how='left')
            .merge(categories_tb, on=['name','address','categories'], how='left')).drop_duplicates()

In [19]:
final_df = final_df.rename(columns={'price_Q_A':'Q_A_Price',
                                    'features_Q_A':'Q_A_Features',
                                    'categories_Q_A': 'Q_A_Categories'})

In [20]:
final_df.head(1)

,name,address,city,state,postal_code,categories,rating,summary,Q_A_loc,Q_A_Info,Q_A_Price,Q_A_Features,Q_A_Categories
0,Billy Wright's Manuel's Style Hot Tamales,Robert E. Lee & Canal Blvd,New Orleans,LA,70124,"Food, Street Vendors",4.5,"Here's a 50-word summary of food, service, only: Manuel's-style tames are a N'awlins classic. Fresh, delicious, & inexpensive. Owners Billy and his wife make them with love, using traditional recipes. Perfect for parties or just a quick bite. Try their tams with beer for an authentic experience. A must-try for anyone visiting NOLA!","<inst>question: Where is Billy Wright's Manuel's Style Hot Tamales located? </inst> Answer: This restaurant is located at Robert E. Lee & Canal Blvd, New Orleans, LA 70124<EOS>","<inst>question: What is good about Billy Wright's Manuel's Style Hot Tamales? </inst> Answer: Here's a 50-word summary of food, service, only: Manuel's-style tames are a N'awlins classic. Fresh, delicious, & inexpensive. Owners Billy and his wife make them with love, using traditional recipes. Perfect for parties or just a quick bite. Try their tams with beer for an authentic experience. A must-try for anyone visiting NOLA!. Food, Street Vendors. This restaurant has a rating of 4.5 stars. It is located in the zip code 70124.<EOS>",<inst>question: How much does the menu cost at the restaurant?</inst> Answer: This restaurant's price range is $<EOS>,"<inst>question: Can you find a vegetarian restaurant near me? (kid-friendly, dog-friendly)</inst> Answer: This restaurant is not specifically vegetarian, but they might have some vegetarian options. It is not specifically kid-friendly and not specifically dog-friendly.<EOS>","<inst>question: Could you recommend me a restuarnt that is known as these keywords - Food, Street Vendors?</inst> Answer: The restaurant that you are looking for is Billy Wright's Manuel's Style Hot Tamales and they are known also as Hot, Tamales, Manuel's, Style, Billy, Wright's, New, Orleans, Tamale, Pie, Cajun, Cuisine, Louisiana, Food, New, Orleans, Cuisine, Restaurant, Food, Service, Restaurant, Review, Restaurant, Recommendation, Delicious, fresh, every, day, Inexpensive, Authentic, Manuel's<EOS>"


In [21]:
# reshape_df = final_df.melt(
#     id_vars=['name', 'address', 'categories'],
#     value_vars=['Q_A_loc', 'Q_A_Info', 'summary'],
#     var_name='source',
#     value_name='text'
# )

# # Map the `source` column to `type_qna`
# reshape_df['type_qna'] = reshape_df['source'].map({
#     'Q_A_loc': 'address_base',
#     'Q_A_Info': 'pos_opinion',
#     'summary' : 'review_summary'
# })

In [22]:
# # Drop unnecessary columns
# reshape_df = reshape_df.drop(columns=['source'])
# reshape_df = reshape_df[['name','address','categories','type_qna','text']]

In [23]:
# # File path to save in the Kaggle datasets directory
# file_path = '/kaggle/working/jaehwan_dataset.csv'

# # Save the DataFrame as a CSV file
# reshape_df.to_csv(file_path, index=False)

# print(f"DataFrame saved to {file_path}")

In [24]:
sample_comb=final_df[['name','Q_A_loc','Q_A_Info','Q_A_Price','Q_A_Features','Q_A_Categories']].sample(n=3027, random_state=42)
sample_comb.head(3)

,name,Q_A_loc,Q_A_Info,Q_A_Price,Q_A_Features,Q_A_Categories
1207,La Carreta,"<inst>question: How can I find La Carreta? </inst> Answer: This restaurant is located at 1814 Magazine St, New Orleans, LA 70130<EOS>","<inst>question: What can you tell me about La Carreta? </inst> Answer: La Carrita is highly recommended for its authentic Mexican dishes, hand-crafted cocktails (including a refreshing margherita), and excellent service, particularly in its outdoor seating area, where it offers a fun and lively atmosphere. The reviewer praises the variety of options, such as their build-your-own burritos, spicy chicken and vegetable soup with homemade tortillitas (tortillas made from scratch in-house), delicious shrimp, fish, steak, chicken, tacos (with various toppings and sauces), f. Restaurants, Mexican. This restaurant has a rating of 4.0 stars. It is located in the zip code 70130.<EOS>",<inst>question: How much does the menu cost at the restaurant?</inst> Answer: This restaurant's price range is $$<EOS>,"<inst>question: Can you find a vegetarian restaurant near me? (kid-friendly, dog-friendly)</inst> Answer: Yes, this is a vegetarian-friendly restaurant! It is not specifically kid-friendly and not specifically dog-friendly.<EOS>","<inst>question: Could you recommend me a restuarnt that is known as these keywords - Restaurants, Mexican?</inst> Answer: The restaurant that you are looking for is La Carreta and they are known also as Mexican, Food, Service, Atmosphere, Quality, Authentic, Flavor, Value, Variety, Fresh, Homemade, Madeinhouse, Hcrafted, Decent, Good, Great, Excellent, Delicious, food, Fresh, ingredients, Homemade, salsa, Homemade<EOS>"
256,Shawarma on the Go,"<inst>question: Where is Shawarma on the Go located? </inst> Answer: This restaurant is located at 3720 Magazine St, New Orleans, LA 70115<EOS>","<inst>question: What is good about Shawarma on the Go? </inst> Answer: Here's an 80-word summary focusing only on ""food and services"" without unnecessary detail: Delicious food, generous portion sizes, high-quality ingredients. Freshly made sauces, meats, vegetables, all cooked to perfection. Friendly, attentive service, with staff knowledgeable about their dishes. Conveniently located, even for those with limited mobility. Affordable prices, making it a gem for quick, satisfying meals on-the-go. Worth a try for any food enthusiast, regardless of location or convenience. Five stars. American (Traditional), Restaurants, Middle Eastern, Mediterranean. This restaurant has a rating of 4.5 stars. It is located in the zip code 70115.<EOS>",<inst>question: How much does the menu cost at the restaurant?</inst> Answer: This restaurant's price range is $<EOS>,"<inst>question: Can you find a vegetarian restaurant near me? (kid-friendly, dog-friendly)</inst> Answer: This restaurant is not specifically vegetarian, but they might have some vegetarian options. It is not specifically kid-friendly and not specifically dog-friendly.<EOS>","<inst>question: Could you recommend me a restuarnt that is known as these keywords - American (Traditional), Restaurants, Middle Eastern, Mediterranean?</inst> Answer: The restaurant that you are looking for is Shawarma on the Go and they are known also as Shawarma, Gyros, Falafel, Hummus, Pita, Bread, Fresh, Food, Authentic, Quality, Value, Quick, Service, Takeout, Food, Truck, Gas, Station, Neighborhood, Staple, Fresh, authentic, food, Highquality, ingredients, preparation<EOS>"
2355,McAlister's Deli,"<inst>question: How can I find McAlister's Deli? </inst> Answer: This restaurant is located at 2701 Airline Hwy, Ste T, Metairie, LA 70001<EOS>","<inst>question: What can you tell me about McAlister's Deli? </inst> Answer: The reviewer has had multiple negative experiences at McAllen's location in Metarie, Louisiana, citing issues with food quality, service, atmosphere, cleanliness, staff behavior, pricing, wait times. They mention that their sandwiches and soups were consistently wrong, including mis

In [25]:
def clean_text(text):
    if pd.isna(text):
        return "Not available"
    return text.replace("<EOS>", "").strip()

# Convert DataFrame to structured JSON
result = []
for _, row in sample_comb.iterrows():
    entry = {
        "name": row["name"],
        "location": clean_text(str(row["Q_A_loc"]).split("Answer:")[1].strip() if "Answer:" in str(row["Q_A_loc"]) else "Not available"),
        "info": clean_text(str(row["Q_A_Info"]).split("Answer:")[1].strip() if "Answer:" in str(row["Q_A_Info"]) else "Not available"),
        "price": clean_text(str(row["Q_A_Price"]).split("Answer:")[1].strip() if "Answer:" in str(row["Q_A_Price"]) else "Not available"),
        "features": clean_text(str(row["Q_A_Features"]).split("Answer:")[1].strip() if "Answer:" in str(row["Q_A_Features"]) else "Not available"),
        "categories": clean_text(str(row["Q_A_Categories"]).split("Answer:")[1].strip() if "Answer:" in str(row["Q_A_Categories"]) else "Not available")
    }
    result.append(entry)

# Convert to JSON string for readability
json_output = json.dumps(result, indent=4)

with open("restaurants.json", "w") as file:
    file.write(json_output)


In [27]:
# Step 1: Preprocess Data (already done in your case)
# def preprocess_data(row):
#     loc_data = row['Q_A_loc']
#     info_data = row['Q_A_Info']
#     return f"{loc_data}\n{info_data}"

# sample_comb['combined'] = sample_comb.apply(preprocess_data, axis=1)
# dataset = sample_comb['combined'].tolist()

In [28]:
!pip install transformers
!pip install datasets
!pip install peft
!pip install bitsandbytes
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -U bitsandbytes
!pip install evaluate
!pip install transformers datasets faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 8.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 20.4 MB/s eta 0:00:00:00:0100:01
Looking in indexes: https://download.pytorch.org/whl/cu118
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 55.9 MB/s eta 0:00:00:00:0100:01


In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    BitsAndBytesConfig
)
from peft import get_peft_model, LoraConfig, PeftModel
import torch
from sklearn.model_selection import train_test_split
import re

# Preprocess Combined Dataset
sample_comb['text'] = (
    sample_comb['name'] + ' ' +
    sample_comb['Q_A_loc'].fillna('') + ' ' +
    sample_comb['Q_A_Info'].fillna('') + ' ' +
    sample_comb['Q_A_Price'].fillna('') + ' ' +
    sample_comb['Q_A_Features'].fillna('') + ' ' +
    sample_comb['Q_A_Categories'].fillna('')
)

# Split the dataset into training and testing sets
train_df, test_df = train_test_split(sample_comb, test_size=0.2, random_state=42)

# Convert to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df[['text']])
test_dataset = Dataset.from_pandas(test_df[['text']])

# Load Tokenizer and Add Special Tokens
model_name = "meta-llama/Llama-3.2-1B"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

if tokenizer.pad_token is None:
    tokenizer.pad_token = "[PAD]"
    tokenizer.add_special_tokens({"pad_token": "[PAD]"})

special_tokens = ["<inst>", "<EOS>"]
if not any(token in tokenizer.additional_special_tokens for token in special_tokens):
    tokenizer.add_special_tokens({'additional_special_tokens': special_tokens})

print("Special tokens:", tokenizer.additional_special_tokens)

# Load Model with Quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

# Resize Token Embeddings to Match Tokenizer
model.resize_token_embeddings(len(tokenizer))

# LoRA Fine-Tuning Configuration
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

# Tokenize Dataset
def tokenize_data(example):
    return tokenizer(
        example['text'],
        padding='max_length',
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )

train_dataset = train_dataset.map(tokenize_data, batched=True, remove_columns=["text"])
test_dataset = test_dataset.map(tokenize_data, batched=True, remove_columns=["text"])

# Add labels for training
def add_labels(dataset):
    return dataset.map(lambda x: {"labels": x["input_ids"]})

train_dataset = add_labels(train_dataset)
test_dataset = add_labels(test_dataset)

# Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    eval_steps=100,
    learning_rate=5e-6,
    weight_decay=0.01,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=16,
    num_train_epochs=5,
    save_steps=500,
    logging_dir="./logs",
    logging_steps=50,
    fp16=True,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    report_to="none"
)

# Trainer Initialization
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Fine-Tune Model
trainer.train()

# Save Model and Tokenizer
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")

# Debugging - Check Alignment
print("Special tokens:", tokenizer.additional_special_tokens)
vocab_size = len(tokenizer)
embedding_size = model.get_input_embeddings().weight.size(0)
print(f"Tokenizer vocabulary size: {vocab_size}")
print(f"Model embedding size: {embedding_size}")

if vocab_size == embedding_size:
    print("The vocabulary size and embedding size match!")
else:
    print(f"Warning: Vocabulary size ({vocab_size}) does not match embedding size ({embedding_size})")

In [ ]:
print(f"Tokenizer vocab size: {len(tokenizer)}")
print(f"Model embedding size before resize: {model.get_input_embeddings().weight.size(0)}")

In [ ]:
# Chatbot
import torch
import re
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# Load tokenizer and add special tokens if necessary
tokenizer = AutoTokenizer.from_pretrained("./fine_tuned_model", trust_remote_code=True)
if '<inst>' not in tokenizer.additional_special_tokens:
    tokenizer.add_special_tokens({'additional_special_tokens': ['<inst>', '<EOS>']})

# Load base model and resize embeddings to match tokenizer vocab size
base_model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)
base_model.resize_token_embeddings(len(tokenizer))

# Load the fine-tuned adapter
model = PeftModel.from_pretrained(base_model, "./fine_tuned_model")

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print(f"Tokenizer vocab size: {len(tokenizer)}")
print(f"Model vocab size: {model.get_input_embeddings().weight.size(0)}")

# Utility function to clean up the model's response
def clean_response(response):
    # Remove all unnecessary artifacts such as <inst>, <EOS>, <PAD>, and <question>
    artifacts = ["<inst>", "<EOS>", "<PAD>", "</inst>", "</a>", "<br>", "<question>", "</question>"]
    for artifact in artifacts:
        response = response.replace(artifact, "")
    
    response = re.sub(r"http[s]?://\S+", "", response)  # Remove URLs
    response = " ".join(response.split())  # Normalize spacing (removes multiple spaces)

    # Return cleaned response
    return response.strip()

# Function to determine max tokens based on question type
def determine_max_tokens(question):
    # Define categories for different types of questions
    if any(keyword in question.lower() for keyword in ["location", "address", "directions"]):
        return 50  # For location-based questions, use fewer tokens
    elif any(keyword in question.lower() for keyword in ["vibe", "atmosphere", "ambiance", "description"]):
        return 100  # For descriptive questions, use more tokens
    else:
        return 70  # Default token limit for other types of questions

# Generate a system prompt for consistent responses (Note: Not used in the actual response)
def generate_system_prompt():
    """
    Returns a system-level instruction for the chatbot (not used in response).
    """
    return (
        "You are an AI assistant trained on a fine-tuned restaurant dataset. "
        "Your task is to provide concise, accurate answers based on the dataset. "
        "Focus on answering questions clearly and succinctly in no more than 3 sentences. "
        "Avoid unnecessary details or generic responses."
    )

# Chatbot response generation function
def chatbot_response(question):
    # Determine the max_tokens based on the question
    max_new_tokens = determine_max_tokens(question)

    # Combine system prompt with user question for clear instruction
    system_prompt = generate_system_prompt()
    
    # The <inst> token is meant to indicate the question part
    input_text = f"{system_prompt}\n<inst>question: {question} </inst>"

    # Tokenize input text, ensuring proper truncation and padding
    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        truncation=True,
        max_length=512,
        padding="max_length"
    ).to(device)

    # Generate response with updated decoding parameters
    output_ids = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        num_beams=3,
        temperature=0.2,
        top_k=40,
        top_p=0.85,
        no_repeat_ngram_size=3,
        early_stopping=True,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

    # Decode the raw output to get the response text
    raw_response = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Clean up the response by removing artifacts and extraneous question repetition
    response = clean_response(raw_response)

    # Ensure we only return the cleaned response without unnecessary parts
    if "question:" in response:
        response = response.split("question:")[1].strip()

    return response

In [ ]:
# Example usage
questions = [
    "Can you give me directions to Billy Wright's Manuel's Style Hot Tamales?",
    "Can you tell me more about the vibe at Mondo?",
    "Are there any signature dishes at Drago's Seafood Restaurant?",
    "How is the service at Ruby Slipper Cafe, and where is it located?",
    "What’s the ambiance like at Antoine's Restaurant, and what’s their address?",
    "What’s a must-try at Maple Street Patisserie?",
    "Are there any vegetarian options at Superior Seafood & Oyster Bar?",
    "Are there any reviews mentioning the food quality at The Blue Flamingo Sports Bar & Grill?",
    "How do I get to Pascal's Manale, and what’s their most popular dish?",
    "How does Shaya compare to other Mediterranean places?"
]

print("Chatbot Responses:\n" + "=" * 50)
for question in questions:
    print(f"Q: {question}")
    response = chatbot_response(question)
    print(f"A: {response}")
    print("-" * 50)

In [ ]:
questions = [
    "Can you give me directions to Billy Wright's Manuel's Style Hot Tamales?",
    "Can you tell me more about the vibe at Mondo?",
    "Are there any signature dishes at Drago's Seafood Restaurant?",
    "How is the service at Ruby Slipper Cafe, and where is it located?",
    "What’s the ambiance like at Antoine's Restaurant, and what’s their address?",
    "What’s a must-try at Maple Street Patisserie?",
    "Are there any vegetarian options at Superior Seafood & Oyster Bar?",
    "Are there any reviews mentioning the food quality at The Blue Flamingo Sports Bar & Grill?",
    "How do I get to Pascal's Manale, and what’s their most popular dish?",
    "How does Shaya compare to other Mediterranean places?"
]
print("Chatbot Responses:\n" + "=" * 50)
for question in questions:
    print(f"Q: {question}")
    response = chatbot_response(question)
    print(f"A: {response}")
    print("-" * 50)